### data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)
generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()
print("-"*70)

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=722.00, M2=1957.00
----------------------------------------------------------------------


### settlement model

In [2]:
set = gp.Model("Settlement") 
set.setParam("PoolSolutions", 15)
set.setParam("PoolSearchMode", 2)
set.setParam("PoolGap", 0.05)

alpha = set.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
beta_plus = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
beta_minus = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
x = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M_y = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
e_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
e_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
M_e = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_e = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

# obj. func.
obj = gp.quicksum(
    P_DA[t] * alpha[t] for t in range(T)
) + gp.quicksum(
    1/S * (P_RT[t, s] * beta_plus[t, s] - P_PN[t] * beta_minus[t, s])
    for t in range(T) for s in range(S)
)

set.setObjective(obj, GRB.MAXIMIZE)

Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20
Set parameter PoolSolutions to value 15
Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0.05


In [3]:
# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t, s] == y_plus[i, t, s] - y_minus[i, t, s])

# rt 조건
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= R[i, t, s])

# alpha, beta+, beta- 결정
for t in range(T):
    for s in range(S):
        set.addConstr(alpha[t] == gp.quicksum(x[i, t, s] for i in range(I)))
        set.addConstr(beta_plus[t,s] == gp.quicksum(e_plus[i, t, s] for i in range(I)))
        set.addConstr(beta_minus[t,s] == gp.quicksum(e_minus[i, t, s] for i in range(I)))


# sum e+, sum e- 결정 지어주기
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t, s] for i in range(I)) 
                    == gp.quicksum(e_plus[i, t, s] for i in range(I)) - gp.quicksum(e_minus[i, t, s] for i in range(I)))

# sum e+ <= sum R
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))

# e+, e- 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                e_plus[i,t,s] == y_plus[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                e_minus[i,t,s] == y_minus[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i)
            )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 본인이 주는 양은 자신의 y+을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )
            # 본인이 받는 양은 자신의 y-을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)
            
# # y+, y- 둘 중 하나는 0이어야함
# for i in range(I):
#     for t in range(T):
#         for s in range(S):
#             set.addConstr(y_plus[i, t, s] <= M_y * z_y[i, t, s])
#             set.addConstr(y_minus[i, t, s] <= M_y * (1 - z_y[i, t, s]))

# # sum e+, sum e- 둘 중 하나는 0이어야 함
# for t in range(T):
#     for s in range(S):
#         set.addConstr(gp.quicksum(e_plus[i, t, s] for i in range(I)) <= M_e * z_e[t, s])
#         set.addConstr(gp.quicksum(e_minus[i, t, s] for i in range(I)) <= M_e * (1 - z_e[t, s]))

In [4]:
set.optimize()

if set.status == GRB.OPTIMAL:
    alpha_vals = np.array([alpha[t].x for t in range(T)])
    beta_plus_vals = np.array(
        [[beta_plus[t, s].x for s in range(S)] for t in range(T)]
    )
    beta_minus_vals = np.array(
        [[beta_minus[t, s].x for s in range(S)] for t in range(T)]
    )
    x_vals = np.array(
        [[[x[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_plus_vals = np.array(
        [[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    y_minus_vals = np.array(
        [[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    d_vals = np.array(
        [
            [[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)]
            for i in range(I)
        ]
    )
    e_plus_vals = np.array(
        [[[e_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    e_minus_vals = np.array(
        [[[e_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)]
    )
    given_vals = np.array(
        [[[sum(d[i, j, t, s].x for j in range(I) if j != i) for s in range(S)] for t in range(T)] for i in range(I)]
    )
    received_vals = np.array(
        [[[sum(d[j, i, t, s].x for j in range(I) if j != i) for s in range(S)] for t in range(T)] for i in range(I)]
    )
    print("\n- - - - - - - - - - - - - - - - - - - - - - - -")
    print("Optimal solution found!")
    print(set.objVal)
    print(f"Optimality Gap: {set.MIPGap}")
else:
    print("\nNo optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  15
PoolSearchMode  2
PoolGap  0.05

Optimize a model with 28800 rows, 41784 columns and 124560 nonzeros
Model fingerprint: 0x6f50645f
Variable types: 37464 continuous, 4320 integer (4320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 23360 rows and 18918 columns
Presolve time: 0.02s
Presolved: 5440 rows, 22866 columns, 38429 nonzeros
Variable types: 18546 continuous, 4320 integer (4320 binary)

Root relaxation: objective 1.567562e+06, 2409 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

* 

### 결과 분석

#### 수익 분석

#### 하루 aggregated 커밋량 분석

In [5]:
for t in range(T):
    avg_alpha = sum(x_vals[i, t, s] for i in range(I) for s in range(S)) / S
    avg_e_plus = sum(e_plus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    avg_e_minus = sum(e_minus_vals[i, t, s] for i in range(I) for s in range(S)) / S
    print(f"[시간 {t}] alpha: {avg_alpha:.3f}, e+: {avg_e_plus:.3f}, e-: {avg_e_minus:.3f}")

print(f"총 하루 commitment: {sum(alpha_vals[t] for t in range(T)):.3f}")

[시간 0] alpha: 0.000, e+: 0.000, e-: 0.000
[시간 1] alpha: 0.000, e+: 0.000, e-: 0.000
[시간 2] alpha: 0.000, e+: 0.000, e-: 0.000
[시간 3] alpha: 0.000, e+: 0.000, e-: 0.000
[시간 4] alpha: 0.000, e+: 0.600, e-: 0.000
[시간 5] alpha: 0.000, e+: 0.000, e-: 0.000
[시간 6] alpha: 2.000, e+: 1.367, e-: 0.100
[시간 7] alpha: 7.000, e+: 5.700, e-: 0.100
[시간 8] alpha: 35.000, e+: 12.067, e-: 0.700
[시간 9] alpha: 143.000, e+: 33.400, e-: 6.833
[시간 10] alpha: 299.000, e+: 144.633, e-: 5.533
[시간 11] alpha: 0.000, e+: 661.033, e-: 0.000
[시간 12] alpha: 0.000, e+: 875.367, e-: 0.000
[시간 13] alpha: 0.000, e+: 1259.700, e-: 0.000
[시간 14] alpha: 1229.000, e+: 213.233, e-: 77.200
[시간 15] alpha: 0.000, e+: 868.267, e-: 0.000
[시간 16] alpha: 636.000, e+: 186.733, e-: 22.033
[시간 17] alpha: 0.000, e+: 763.667, e-: 0.000
[시간 18] alpha: 0.000, e+: 542.167, e-: 0.000
[시간 19] alpha: 291.000, e+: 108.300, e-: 14.067
[시간 20] alpha: 90.000, e+: 17.533, e-: 3.800
[시간 21] alpha: 13.000, e+: 6.100, e-: 1.033
[시간 22] alpha: 0.000, e

#### exchange process

In [6]:
for t in range(19,22):
    for s in range(1,4):
        x_sum = sum(x_vals[i, t, s] for i in range(I))
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))

        e_plus_sum = sum(e_plus_vals[i, t, s] for i in range(I))
        e_minus_sum = sum(e_minus_vals[i, t, s] for i in range(I))

        R_sum = sum(R[i, t, s] for i in range(I))

        print(f"[t={t} s={s}]")

        print(f"R 합계: {R_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {R[i, t, s]:.2f}", end=" ")
        print(")")
        
        print(f"alpha: {alpha_vals[t]:.2f} ", end="")
        print(f"(", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i, t, s]:.2f}", end=" ")
        print(")")

        print(f"y+ 합계: {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y- 합계: {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")
        
        print(f"beta+: {beta_plus_vals[t,s]:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"beta-: {beta_minus_vals[t,s]:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {e_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print("거래 내역:")
        for i in range(I):
            for j in range(I):
                if d_vals[i, j, t, s] > 1e-6:
                    print(
                        f"발전기 {i}가 발전기 {j}에게 {d_vals[i,j,t,s]:.2f} 전력을 줌"
                    )
        print()

[t=19 s=1]
R 합계: 417.00 ([0] 88.00 [1] 87.00 [2] 56.00 [3] 158.00 [4] 28.00 )
alpha: 291.00 ([0] 291.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
y+ 합계: 329.00 ([0] 0.00 [1] 87.00 [2] 56.00 [3] 158.00 [4] 28.00 )
y- 합계: 203.00 ([0] 203.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
beta+: 126.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 126.00 [4] 0.00 )
beta-: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
거래 내역:
발전기 1가 발전기 0에게 87.00 전력을 줌
발전기 2가 발전기 0에게 56.00 전력을 줌
발전기 3가 발전기 0에게 32.00 전력을 줌
발전기 4가 발전기 0에게 28.00 전력을 줌

[t=19 s=2]
R 합계: 248.00 ([0] 86.00 [1] 40.00 [2] 33.00 [3] 61.00 [4] 28.00 )
alpha: 291.00 ([0] 291.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
y+ 합계: 162.00 ([0] 0.00 [1] 40.00 [2] 33.00 [3] 61.00 [4] 28.00 )
y- 합계: 205.00 ([0] 205.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
beta+: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
beta-: 43.00 ([0] 43.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 )
거래 내역:
발전기 1가 발전기 0에게 40.00 전력을 줌
발전기 2가 발전기 0에게 33.00 전력을 줌
발전기 3가 발전기 0에게 61.00 전력을 줌
발전기 4가

### 정산

In [7]:
# 시간별 Day-ahead 수익 계산
hourly_da = np.zeros(T)
for t in range(T):
    hourly_da[t] = P_DA[t] * alpha[t].x

# 전체 Day-ahead 수익
daily_da = np.sum(hourly_da)


# 시간별 Real-time 수익 계산
hourly_rt = np.zeros(T)
for t in range(T):
    hourly_rt[t] = sum(
        (1 / S) * sum(P_RT[t, s] * e_plus[i, t, s].x for s in range(S)) for i in range(I)
    )

# 전체 Real-time 수익
daily_rt = np.sum(hourly_rt)


# 시간별 패널티 비용 계산
hourly_pen = np.zeros(T)
for t in range(T):
    hourly_pen[t] = sum(
        (1 / S) * sum(P_PN[t] * e_minus[i, t, s].x for s in range(S)) for i in range(I)
    )

# 전체 패널티 비용
daily_pen = np.sum(hourly_pen)


# 시간별 총 시스템 이익 계산
hourly_system_profit = hourly_da + hourly_rt - hourly_pen

# 전체 총 시스템 이익
daily_system_profit = daily_da + daily_rt - daily_pen

# # 📌 결과 출력
# print("🔹 Hourly Profits (by t):")
# for t in range(T):
#     print(f"[t={t:02d}] DA: {hourly_da[t]:.2f}, RT: {hourly_rt[t]:.2f}, "
#           f"Penalty: {hourly_pen[t]:.2f}, Total Profit: {hourly_system_profit[t]:.2f}")

print("\n🔹 Total Profits (summed over all t):")
print(f"Total DA Profit: {daily_da:.2f}")
print(f"Total RT Profit: {daily_rt:.2f}")
print(f"Total Penalty Cost: {daily_pen:.2f}")
print(f"Total System Profit: {daily_system_profit:.2f}")


🔹 Total Profits (summed over all t):
Total DA Profit: 418449.85
Total RT Profit: 1179399.40
Total Penalty Cost: 30287.36
Total System Profit: 1567561.88


In [8]:
nine = hourly_contribution(x_vals + e_plus_vals + given_vals - e_minus_vals - received_vals)
nineh, nined = remuneration(nine, hourly_system_profit)

ten = hourly_contribution(x_vals + y_plus_vals - y_minus_vals)
tenh, tend = remuneration(ten, hourly_system_profit)

eleven = hourly_contribution(R)
elevenh, elevend = remuneration(eleven, hourly_system_profit)

twelve_da = hourly_contribution(x_vals)
twelve_rt = hourly_contribution(e_plus_vals + given_vals)
twelve_pn = hourly_contribution(e_minus_vals + received_vals)
twelve = hourly_contribution(twelve_da + twelve_rt + 1/(twelve_pn+0.000001))
twelveh_da, twelved_da = remuneration(twelve_da, hourly_da)
twelveh_rt, twelved_rt = remuneration(twelve_rt, hourly_rt)
twelveh_pn, twelved_pn = remuneration(twelve_pn, hourly_pen)
twelveh = twelveh_da+twelveh_rt-twelveh_pn
twelved = twelved_da+twelved_rt-twelved_pn

thirteen = hourly_contribution(x_vals+y_plus_vals)
thirteenh, thirteend = remuneration(thirteen, hourly_system_profit)

w1 = daily_contribution(x_vals + e_plus_vals + given_vals - e_minus_vals - received_vals)
w1h, w1d = remuneration(w1, daily_system_profit)

w2 = daily_contribution(
    x_vals * P_DA[np.newaxis, :, np.newaxis]
    + e_plus_vals * P_RT
    + given_vals * P_RT
    - e_minus_vals * P_PN[np.newaxis, :, np.newaxis]
    - received_vals * P_PN[np.newaxis, :, np.newaxis]
)
w2h, w2d = remuneration(w2, daily_system_profit)

w3 = daily_contribution(R)
w3h, w3d = remuneration(w3, daily_system_profit)

w4 = daily_contribution(R*P_DA[np.newaxis, :, np.newaxis])
w4h, w4d = remuneration(w4, daily_system_profit)

xx_da = daily_contribution(x_vals)
xx_rt = daily_contribution(y_plus_vals)
xx_pn = daily_contribution(y_minus_vals)
xxh_da, xxd_da = remuneration(xx_da, daily_da)
xxh_rt, xxd_rt = remuneration(xx_rt, daily_rt)
xxh_pn, xxd_pn = remuneration(xx_pn, daily_pen)
xxh = xxh_da + xxh_rt - xxh_pn
xxd = xxd_da + xxd_rt - xxd_pn

plot_daily_remuneration(elevend, twelved, w1d, w2d, w3d, w4d, xxd, labels=["11", "12", "w1", "w2", "w3", "w4", "xx"])

NameError: name 'hourly_contribution' is not defined